In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from czsc import KlineAnalyze

import tushare as ts
ts.set_token('14f4fbfa08778cba51c73f0ef928210cf2629931fb06f3a797564a0f')

定义数据获取程序如下（实际情况根据需要进行开发）：

In [2]:
from datetime import datetime, timedelta

def get_kline(ts_code, start_date,end_date, freq='30min'):
    """获取指定级别的前复权K线

    :param ts_code: str
        股票代码，如 600122.SH
    :param freq: str
        K线级别，可选值 [1min, 5min, 15min, 30min, 60min, D, M, Y]
    :param start_date,end_date: str
        日期，如 20190610
    :return: pd.DataFrame
        columns = ["symbol", "dt", "open", "close", "high", "low", "vol"]
    """

    end_date = datetime.strptime(end_date, '%Y%m%d')
    end_date = end_date + timedelta(days=1)
    end_date = end_date.date().__str__().replace("-", "")

    pro = ts.pro_api()
    df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
    #print(df)

    # 统一 k 线数据格式为 6 列，分别是 ["symbol", "dt", "open", "close", "high", "low", "vr"]
    if "min" in freq:
        df.rename(columns={'ts_code': "symbol", "trade_time": "dt"}, inplace=True)
    else:
        df.rename(columns={'ts_code': "symbol", "trade_date": "dt"}, inplace=True)

    df.drop_duplicates(subset='dt', keep='first', inplace=True)
    df.sort_values('dt', inplace=True)
    df['dt'] = df.dt.apply(str)
    if freq.endswith("min"):
        # 清理 9:30 的空数据
        df['not_start'] = df.dt.apply(lambda x: not x.endswith("09:30:00"))
        df = df[df['not_start']]
    df.reset_index(drop=True, inplace=True)

    k = df[['symbol', 'dt', 'open', 'close', 'high', 'low', 'vol']]

    for col in ['open', 'close', 'high', 'low']:
        k[col] = k[col].apply(round, args=(2,))
    return k

In [3]:
kline = get_kline(ts_code="600584.SH", start_date="20170710",end_date="20200710", freq='D')
kline

/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,symbol,dt,open,close,high,low,vol
0,600584.SH,20170710,17.14,16.75,17.14,16.59,189919.24
1,600584.SH,20170711,16.67,16.02,16.68,16.00,208657.76
2,600584.SH,20170712,16.00,15.98,16.18,15.60,143915.65
3,600584.SH,20170713,16.00,15.64,16.07,15.58,121048.21
4,600584.SH,20170714,15.63,15.95,16.03,15.35,169544.33
...,...,...,...,...,...,...,...
719,600584.SH,20200706,35.90,37.51,37.51,35.08,1240966.75
720,600584.SH,20200707,40.00,38.34,41.26,38.00,2030649.43
721,600584.SH,20200708,38.94,41.62,42.00,37.71,1676926.20
722,600584.SH,20200709,40.91,43.33,44.39,40.29,1417807.11


使用 KlineAnalyze 进行分析

In [4]:
ka = KlineAnalyze(kline)

In [5]:
ka.to_html()

/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
/Users/zhushuqiong/anaconda/envs/abu/lib/python3.7/site-packages/pyecharts/charts/chart.py:14: PendingDeprecationWarning: pyecha

In [6]:
ka.up_zs_number(),ka.down_zs_number()

(0, 0)

In [8]:
ka.zs_list

[{'ZD': 15.01,
  'ZG': 16.17,
  'G': 16.17,
  'GG': 26.12,
  'D': 16.42,
  'DD': 8.04,
  'points': [{'dt': '20170718',
    'fx_mark': 'd',
    'fx_high': 15.75,
    'fx_low': 15.01,
    'xd': 15.01},
   {'dt': '20170720',
    'fx_mark': 'g',
    'fx_high': 16.17,
    'fx_low': 15.45,
    'xd': 16.17},
   {'dt': '20170811',
    'fx_mark': 'd',
    'fx_high': 15.08,
    'fx_low': 14.82,
    'xd': 14.82},
   {'dt': '20171113',
    'fx_mark': 'g',
    'fx_high': 26.12,
    'fx_low': 23.95,
    'xd': 26.12},
   {'dt': '20180207',
    'fx_mark': 'd',
    'fx_high': 17.45,
    'fx_low': 16.42,
    'xd': 16.42},
   {'dt': '20180425',
    'fx_mark': 'g',
    'fx_high': 24.45,
    'fx_low': 23.05,
    'xd': 24.45},
   {'dt': '20190104',
    'fx_mark': 'd',
    'fx_high': 8.37,
    'fx_low': 8.04,
    'xd': 8.04},
   {'dt': '20190312',
    'fx_mark': 'g',
    'fx_high': 16.99,
    'fx_low': 14.88,
    'xd': 16.99},
   {'dt': '20190507',
    'fx_mark': 'd',
    'fx_high': 11.21,
    'fx_low': 10.5